## Portfolio Exercise: Starbucks
<br>

<img src="https://opj.ca/wp-content/uploads/2018/02/New-Starbucks-Logo-1200x969.jpg" width="200" height="200">
<br>
<br>
 
#### Background Information

The dataset you will be provided in this portfolio exercise was originally used as a take-home assignment provided by Starbucks for their job candidates. The data for this exercise consists of about 120,000 data points split in a 2:1 ratio among training and test files. In the experiment simulated by the data, an advertising promotion was tested to see if it would bring more customers to purchase a specific product priced at $10. Since it costs the company 0.15 to send out each promotion, it would be best to limit that promotion only to those that are most receptive to the promotion. Each data point includes one column indicating whether or not an individual was sent a promotion for the product, and one column indicating whether or not that individual eventually purchased that product. Each individual also has seven additional features associated with them, which are provided abstractly as V1-V7.

#### Optimization Strategy

Your task is to use the training data to understand what patterns in V1-V7 to indicate that a promotion should be provided to a user. Specifically, your goal is to maximize the following metrics:

* **Incremental Response Rate (IRR)** 

IRR depicts how many more customers purchased the product with the promotion, as compared to if they didn't receive the promotion. Mathematically, it's the ratio of the number of purchasers in the promotion group to the total number of customers in the purchasers group (_treatment_) minus the ratio of the number of purchasers in the non-promotional group to the total number of customers in the non-promotional group (_control_).

$$ IRR = \frac{purch_{treat}}{cust_{treat}} - \frac{purch_{ctrl}}{cust_{ctrl}} $$


* **Net Incremental Revenue (NIR)**

NIR depicts how much is made (or lost) by sending out the promotion. Mathematically, this is 10 times the total number of purchasers that received the promotion minus 0.15 times the number of promotions sent out, minus 10 times the number of purchasers who were not given the promotion.

$$ NIR = (10\cdot purch_{treat} - 0.15 \cdot cust_{treat}) - 10 \cdot purch_{ctrl}$$

For a full description of what Starbucks provides to candidates see the [instructions available here](https://drive.google.com/open?id=18klca9Sef1Rs6q8DW4l7o349r8B70qXM).

Below you can find the training data provided.  Explore the data and different optimization strategies.

#### How To Test Your Strategy?

When you feel like you have an optimization strategy, complete the `promotion_strategy` function to pass to the `test_results` function.  
From past data, we know there are four possible outomes:

Table of actual promotion vs. predicted promotion customers:  

<table>
<tr><th></th><th colspan = '2'>Actual</th></tr>
<tr><th>Predicted</th><th>Yes</th><th>No</th></tr>
<tr><th>Yes</th><td>I</td><td>II</td></tr>
<tr><th>No</th><td>III</td><td>IV</td></tr>
</table>

The metrics are only being compared for the individuals we predict should obtain the promotion – that is, quadrants I and II.  Since the first set of individuals that receive the promotion (in the training set) receive it randomly, we can expect that quadrants I and II will have approximately equivalent participants.  

Comparing quadrant I to II then gives an idea of how well your promotion strategy will work in the future. 

Get started by reading in the data below.  See how each variable or combination of variables along with a promotion influences the chance of purchasing.  When you feel like you have a strategy for who should receive a promotion, test your strategy against the test dataset used in the final `test_results` function.

In [1]:
# load in packages
from itertools import combinations

from test_results import test_results, score
import numpy as np
import pandas as pd
import scipy as sp
import sklearn as sk

import matplotlib.pyplot as plt
import seaborn as sb

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

# hyperparameter search library
from sklearn.model_selection import GridSearchCV

# scoring libraries
from sklearn.metrics import make_scorer, f1_score, classification_report

%matplotlib inline


# load in the data
train_data = pd.read_csv('./training.csv')
train_data.head(10)

,ID,Promotion,purchase,V1,V2,V3,V4,V5,V6,V7
0,1,No,0,2,30.443518,-1.165083,1,1,3,2
1,3,No,0,3,32.159350,-0.645617,2,3,2,2
2,4,No,0,2,30.431659,0.133583,1,1,4,2
3,5,No,0,0,26.588914,-0.212728,2,1,4,2
4,8,Yes,0,3,28.044331,-0.385883,1,1,2,2
5,9,No,0,1,41.189415,-0.905350,2,4,4,1
6,11,No,0,1,40.690409,1.085939,2,2,1,2
7,14,No,0,2,25.971529,-1.424817,1,3,3,2
8,15,Yes,0,2,31.930423,0.393317,2,3,1,2
9,16,No,0,1,27.474650,0.566472,2,2,3,2


In [2]:
# create copy of training data and clean ID
train_data2=train_data.copy()
train_data2.drop(columns=["ID"],inplace=True)  # Kick out ID

# Replace "Yes" and "No" in "Promotions" column with 1 and 0, respectively
f=lambda x : (1 if x=="Yes" else 0)
train_data2.Promotion=np.array([f(train_data2.Promotion.iloc[i]) for i in range(0,len(train_data2))])


In [23]:
# create subsets for all combinations of "Promotion", "purchase"

subsetno=180000
train_data_sure_buyer=sk.utils.shuffle(train_data2[(train_data2.purchase==1) & (train_data2.Promotion==0)]).head(subsetno)
train_data_sure_not_buyer=sk.utils.shuffle(train_data2[(train_data2.purchase==0) & (train_data2.Promotion==1)]).head(subsetno)
train_data_promoted_buyer=sk.utils.shuffle(train_data2[(train_data2.purchase==1) & (train_data2.Promotion==1)]).head(subsetno)
train_data_unpromoted_not_buyer=sk.utils.shuffle(train_data2[(train_data2.purchase==0) & (train_data2.Promotion==0)]).head(subsetno)


# adjust dataframes
train_data_sure_buyer.drop(columns=['Promotion','purchase'],inplace=True)
train_data_sure_buyer['parameter']=0

train_data_sure_not_buyer.drop(columns=['Promotion','purchase'],inplace=True)
train_data_sure_not_buyer['parameter']=0

train_data_promoted_buyer.drop(columns=['Promotion','purchase'],inplace=True)
train_data_promoted_buyer['parameter']=1

train_data_unpromoted_not_buyer.drop(columns=['Promotion','purchase'],inplace=True)
train_data_unpromoted_not_buyer['parameter']=1



In [24]:
print(train_data_sure_buyer.shape)
print(train_data_sure_not_buyer.shape)

print(train_data_promoted_buyer.shape)

print(train_data_unpromoted_not_buyer.shape)

(319, 8)
(41643, 8)
(721, 8)
(41851, 8)


In [25]:
# create 2 more subsets in which those who do not need a promotion have a value of zero and those who need or perhaps (we do not know) may not need a promotion get value 1.

train_data_sure_buyer_and_unpromoted_not_buyer=pd.concat([train_data_sure_buyer, train_data_unpromoted_not_buyer])
train_data_sure_not_buyer_and_promoted_buyer=pd.concat([train_data_sure_not_buyer, train_data_promoted_buyer])

In [26]:
# build train and test set for train_data_sure_buyer_and_unpromoted_not_buyer
X_train=train_data_sure_buyer_and_unpromoted_not_buyer.iloc[:,:7]
y_train=np.array(train_data_sure_buyer_and_unpromoted_not_buyer.iloc[:,7:8].values).ravel()

# build train and test set for train_data_sure_not_buyer_and_promoted_buyer
X_train2=train_data_sure_not_buyer_and_promoted_buyer.iloc[:,:7]
y_train2=np.array(train_data_sure_not_buyer_and_promoted_buyer.iloc[:,7:8].values).ravel()

In [217]:
# classifier libraries
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

# library for estimator base class
from sklearn.base import BaseEstimator

def combine_probabilities_into_output(prob_a,prob_b):
    """
    It is aimed to combine two preditor probabilities into one
    
    INPUT
    prob_a - probability of event a
    prob_b - probability of event b
    
    OUTPUT
    "YES" or "NO" depending on the combined condition
    """
    # if ((prob_a>0.5) & (prob_b>0.5)):
    if ((prob_a+prob_b>1.0)&(prob_a>0.43) & (prob_b>0.565)):
        return "Yes"
    else:
        return "No"

class own_classifier(BaseEstimator):
    
    def __init__(self, tempi=0, estimator=[LogisticRegression(),LogisticRegression()], C_param=100):
        """constructor function"""
        print("konstruktor: ", estimator)
        
        self.set_params(tempi,estimator, C_param)
        return None

    def set_params(self, tempi=0,estimator=[LogisticRegression(),LogisticRegression()], C_param=100):
        """parameters are set"""
        
        print("zuerst: ",estimator)
        self.estimator = estimator# [estimator[0],estimator[1]]
        self.C_param = C_param
        self.classes_ = [0, 1]
        self.tempi=tempi
        print(self.estimator)
        self.estimator[0].set_params( random_state=0, C=self.C_param, max_iter=5000, class_weight="balanced" )
        self.estimator[1].set_params( random_state=0, C=self.C_param, max_iter=5000, class_weight="balanced" )
        # self.model = [ self.estimator[0],self.estimator[1]]
        print("in set_params proc", self.tempi, "fghjk")
        return self
    
    def combine_probabilities_into_output(prob_a,prob_b):
        """
        It is aimed to combine two preditor probabilities into one
        
        INPUT
        prob_a - probability of event a
        prob_b - probability of event b
        
        OUTPUT
        "YES" or "NO" depending on the combined condition
        """
        # if ((prob_a>0.5) & (prob_b>0.5)):
        if ((prob_a+prob_b>1.0)&(prob_a>0.43) & (prob_b>0.565)):
            return "Yes"
        else:
            return "No"

    def score(self, X, y):
       """scoring function"""
       return self.model[0].score(X[0], y[0])+self.model[1].score(X[1], y[1])
    #def __sklearn_clone__(self):    
    #    return self
    def fit(self,X2,y=None):
        """fit procedure covering all classifiers for each target"""

        X=X2.copy()
        train_data_sure_buyer=sk.utils.shuffle(X[(X.purchase==1) & (X.Promotion==0)])
        train_data_sure_not_buyer=sk.utils.shuffle(X[(X.purchase==0) & (X.Promotion==1)])
        train_data_promoted_buyer=sk.utils.shuffle(X[(X.purchase==1) & (X.Promotion==1)])
        train_data_unpromoted_not_buyer=sk.utils.shuffle(X[(X.purchase==0) & (X.Promotion==0)])


        # adjust dataframes
        train_data_sure_buyer.drop(columns=['Promotion','purchase'],inplace=True)
        train_data_sure_buyer['parameter']=0

        train_data_sure_not_buyer.drop(columns=['Promotion','purchase'],inplace=True)
        train_data_sure_not_buyer['parameter']=0

        train_data_promoted_buyer.drop(columns=['Promotion','purchase'],inplace=True)
        train_data_promoted_buyer['parameter']=1

        train_data_unpromoted_not_buyer.drop(columns=['Promotion','purchase'],inplace=True)
        train_data_unpromoted_not_buyer['parameter']=1


 
        print("in fit proc")
        
        # print(len(y))
        # print(y)
        
        
        # self.model = [ self.estimator[0]( random_state=0, C=self.C_param, max_iter=5000, class_weight="balanced" ),  self.estimator[1](  random_state=0, C=self.C_param, max_iter=5000, class_weight="balanced"  )     ]
        # self.model[0].fit(X[0], y[0])
        # self.model[1].fit(X[1], y[1])

        # create 2 more subsets in which those who do not need a promotion have a value of zero and those who need or perhaps (we do not know) may not need a promotion get value 1.

        train_data_sure_buyer_and_unpromoted_not_buyer=pd.concat([train_data_sure_buyer, train_data_unpromoted_not_buyer])
        train_data_sure_not_buyer_and_promoted_buyer=pd.concat([train_data_sure_not_buyer, train_data_promoted_buyer])

        # build train and test set for train_data_sure_buyer_and_unpromoted_not_buyer
        X_train=train_data_sure_buyer_and_unpromoted_not_buyer[['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7']]
        y_train=np.array(train_data_sure_buyer_and_unpromoted_not_buyer[['parameter']].values).ravel()

        # build train and test set for train_data_sure_not_buyer_and_promoted_buyer
        X_train2=train_data_sure_not_buyer_and_promoted_buyer[['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7']]
        y_train2=np.array(train_data_sure_not_buyer_and_promoted_buyer[['parameter']].values).ravel()
        self.estimator[0].fit(X_train,y_train)
        self.estimator[1].fit(X_train2,y_train2)

        return self

    # def fit(self, X, y=None):
    #     """fit procedure covering all classifiers for each target"""
    #     f1 = y.sum() / y.shape[0]
    #     if f1 == 0:
    #         self.one_class_only = True
    #         self.one_class_value = 0
    #         return self
    #     elif f1 == 1:
    #         self.one_class_only = True
    #         self.one_class_value = 1
    #         return self
    #     else:
    #         self.one_class_only = False

    #     self.model = OneVsRestClassifier(
    #         self.estimator(
    #             random_state=0, C=self.C_param, max_iter=5000, class_weight="balanced"
    #         ),
    #         n_jobs=-1
    #     )
    #     self.model.fit(X, y)
    #     return self

    def predict(self, X):
        """predict-funtion for the only one target value"""
        # if self.one_class_only:
        #     return [self.one_class_value for i in range(X.shape[0])]
        #print(X.columns)
        # X=X2.copy()
        # X.drop(columns=['Promotion','purchase'],inplace=True)
        y_pred = self.estimator[0].predict_proba(X.iloc[:,:]).T[1]
        y_pred2 = self.estimator[1].predict_proba(X.iloc[:,:]).T[1]
        print(y_pred)
        print(y_pred2)
        return np.array(list(map(combine_probabilities_into_output,y_pred,y_pred2)))

In [218]:
# tr=own_classifier()
own_classifier().fit(train_data2)
own_classifier().predict(train_data2.iloc[0:5,2:9])
# own_classifier().fit(X_train, y_train)
print("hjk")
train_data2

konstruktor:  [LogisticRegression(), LogisticRegression()]
zuerst:  [LogisticRegression(), LogisticRegression()]
[LogisticRegression(), LogisticRegression()]
in set_params proc 0 fghjk
in fit proc
konstruktor:  [LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0), LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0)]
zuerst:  [LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0), LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0)]
[LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0), LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0)]
in set_params proc 0 fghjk
[0.55499312 0.52833234 0.52657368 0.4775765  0.5536367 ]
[0.27674068 0.56908162 0.24389393 0.54537198 0.25055352]
hjk


,Promotion,purchase,V1,V2,V3,V4,V5,V6,V7
0,0,0,2,30.443518,-1.165083,1,1,3,2
1,0,0,3,32.159350,-0.645617,2,3,2,2
2,0,0,2,30.431659,0.133583,1,1,4,2
3,0,0,0,26.588914,-0.212728,2,1,4,2
4,1,0,3,28.044331,-0.385883,1,1,2,2
...,...,...,...,...,...,...,...,...,...
84529,0,0,1,30.084876,1.345672,1,1,3,1
84530,1,0,3,33.501485,-0.299306,1,1,4,1
84531,0,0,1,31.492019,1.085939,2,3,2,2
84532,0,0,1,37.766106,0.999361,2,2,1,2


In [219]:
def build_model():
    """Definition of the model via pipeline and a parameters variable.
    The best parameters were determined using grid search.
    In this example, the bag of words approach and the word-to-vetor approach are
    """

    pipeline= Pipeline([
        ('clf',own_classifier())
    ])


    parameters = [
        {
            #"clf__estimator": ([LogisticRegression,LogisticRegression])
            "clf__tempi": ([1])
        }
    ]

    cv = GridSearchCV(
        pipeline,
        param_grid=parameters,
        refit=True,
        # scoring=make_scorer(f1_score, **dict(average="macro", pos_label=1, zero_division=0))
    )
    return cv

In [220]:
# define and fit pipelines



# # build pipeline
# pipeline= Pipeline([
#     ('clf',LogisticRegression())
# ])
# pipeline2= Pipeline([
#     ('clf',LogisticRegression())
# ])

# ######



#fit pipelines
cv=build_model()
print("hshs ",cv.estimator)
print("  ")
cv.fit(train_data2)
# pipeline2.fit(X_train2,y_train2)


konstruktor:  [LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0), LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0)]
zuerst:  [LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0), LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0)]
[LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0), LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0)]
in set_params proc 0 fghjk
hshs  Pipeline(steps=[('clf', own_classifier())])
  
konstruktor:  [LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0), LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0)]
zuerst:  [LogisticRegression(C=100, class_weight='balanced', ma

g:\python\Lib\site-packages\sklearn\model_selection\_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "g:\python\Lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "g:\python\Lib\site-packages\sklearn\metrics\_scorer.py", line 444, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "g:\python\Lib\site-packages\sklearn\pipeline.py", line 722, in score
    return self.steps[-1][1].score(Xt, y, **score_params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\omf2\AppData\Local\Temp\ipykernel_19476\4167347458.py", line 68, in score
    return self.model[0].score(X[0], y[0])+self.model[1].score(X[1], y[1])
           ^^^^^^^^^^
AttributeError: 'own_classifier'

konstruktor:  [LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0), LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0)]
zuerst:  [LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0), LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0)]
[LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0), LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0)]
in set_params proc 0 fghjk
zuerst:  [LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0), LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0)]
[LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0), LogisticRegression

g:\python\Lib\site-packages\sklearn\model_selection\_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "g:\python\Lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "g:\python\Lib\site-packages\sklearn\metrics\_scorer.py", line 444, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "g:\python\Lib\site-packages\sklearn\pipeline.py", line 722, in score
    return self.steps[-1][1].score(Xt, y, **score_params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\omf2\AppData\Local\Temp\ipykernel_19476\4167347458.py", line 68, in score
    return self.model[0].score(X[0], y[0])+self.model[1].score(X[1], y[1])
           ^^^^^^^^^^
AttributeError: 'own_classifier'

konstruktor:  [LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0), LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0)]
zuerst:  [LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0), LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0)]
[LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0), LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0)]
in set_params proc 0 fghjk
zuerst:  [LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0), LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0)]
[LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0), LogisticRegression

...

g:\python\Lib\site-packages\sklearn\model_selection\_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "g:\python\Lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "g:\python\Lib\site-packages\sklearn\metrics\_scorer.py", line 444, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "g:\python\Lib\site-packages\sklearn\pipeline.py", line 722, in score
    return self.steps[-1][1].score(Xt, y, **score_params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\omf2\AppData\Local\Temp\ipykernel_19476\4167347458.py", line 68, in score
    return self.model[0].score(X[0], y[0])+self.model[1].score(X[1], y[1])
           ^^^^^^^^^^
AttributeError: 'own_classifier'

konstruktor:  [LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0), LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0)]
zuerst:  [LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0), LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0)]
[LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0), LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0)]
in set_params proc 0 fghjk
zuerst:  [LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0), LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0)]
[LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0), LogisticRegression

g:\python\Lib\site-packages\sklearn\model_selection\_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "g:\python\Lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "g:\python\Lib\site-packages\sklearn\metrics\_scorer.py", line 444, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "g:\python\Lib\site-packages\sklearn\pipeline.py", line 722, in score
    return self.steps[-1][1].score(Xt, y, **score_params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\omf2\AppData\Local\Temp\ipykernel_19476\4167347458.py", line 68, in score
    return self.model[0].score(X[0], y[0])+self.model[1].score(X[1], y[1])
           ^^^^^^^^^^
AttributeError: 'own_classifier'

konstruktor:  [LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0), LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0)]
zuerst:  [LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0), LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0)]
[LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0), LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0)]
in set_params proc 0 fghjk
zuerst:  [LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0), LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0)]
[LogisticRegression(C=100, class_weight='balanced', max_iter=5000,
                   random_state=0), LogisticRegression

GridSearchCV(estimator=Pipeline(steps=[('clf', own_classifier())]),
             param_grid=[{'clf__tempi': [1]}])

In [221]:
def promotion_strategy(df):
    '''
    INPUT
    df - a dataframe with *only* the columns V1 - V7 (same as train_data)

    OUTPUT
    promotion_df - np.array with the values
                   'Yes' or 'No' related to whether or not an
                   individual should recieve a promotion
                   should be the length of df.shape[0]

    Ex:
    INPUT: df

    V1	V2	  V3	V4	V5	V6	V7
    2	30	-1.1	1	1	3	2
    3	32	-0.6	2	3	2	2
    2	30	0.13	1	1	4	2

    OUTPUT: promotion

    array(['Yes', 'Yes', 'No'])
    indicating the first two users would recieve the promotion and
    the last should not.
    '''
    print(df.columns)


    

    return cv.predict(df)

In [222]:
# This will test your results, and provide you back some information 
# on how well your promotion_strategy will work in practice

test_results(promotion_strategy)

Index(['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7'], dtype='object')
[0.47028547 0.46250006 0.51778756 ... 0.48797155 0.53971578 0.44025256]
[0.21856772 0.53414189 0.62819958 ... 0.49044455 0.27178923 0.25453106]
['No' 'No' 'Yes' ... 'No' 'No' 'No']
2        Yes
3         No
5        Yes
9         No
10        No
        ... 
41637     No
41638    Yes
41640    Yes
41643     No
41646    Yes
Name: Promotion, Length: 12704, dtype: object
ghjK 6388
ghjK 46
0.01892312731717125
Nice job!  See how well your strategy worked on our test data below!

Your irr with this strategy is 0.0189.

Your nir with this strategy is 242.60.
We came up with a model with an irr of 0.0188 and an nir of 189.45 on the test set.

 How did you do?


(0.01892312731717125, 242.60000000000002)